In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
# (Making sure) Set backend as tensorflow
from keras import backend as K


Using TensorFlow backend.


In [2]:
K.set_image_dim_ordering('tf')

In [3]:
# Define some variables
num_rows = 28
num_cols = 28
num_channels = 1
num_classes = 10
# Import data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], num_rows, num_cols, num_channels).astype(np.float32) / 255
X_test = X_test.reshape(X_test.shape[0], num_rows, num_cols, num_channels).astype(np.float32) / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

11419648/11490434 [============================>.] - ETA: 0s

In [4]:
# Model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
68s - loss: 0.6060 - acc: 0.7999 - val_loss: 0.1283 - val_acc: 0.9638
Epoch 2/10
67s - loss: 0.1798 - acc: 0.9431 - val_loss: 0.0832 - val_acc: 0.9768
Epoch 3/10
67s - loss: 0.1292 - acc: 0.9592 - val_loss: 0.0655 - val_acc: 0.9820
Epoch 4/10
62s - loss: 0.1086 - acc: 0.9659 - val_loss: 0.0495 - val_acc: 0.9855
Epoch 5/10
62s - loss: 0.0938 - acc: 0.9705 - val_loss: 0.0466 - val_acc: 0.9868
Epoch 6/10
62s - loss: 0.0828 - acc: 0.9734 - val_loss: 0.0415 - val_acc: 0.9881
Epoch 7/10
62s - loss: 0.0781 - acc: 0.9748 - val_loss: 0.0398 - val_acc: 0.9874
Epoch 8/10
62s - loss: 0.0683 - acc: 0.9783 - val_loss: 0.0349 - val_acc: 0.9885
Epoch 9/10
62s - loss: 0.0651 - acc: 0.9795 - val_loss: 0.0331 - val_acc: 0.9897
Epoch 10/10
62s - loss: 0.0614 - acc: 0.9804 - val_loss: 0.0320 - val_acc: 0.9904


In [7]:
# Prepare model for inference
for k in model.layers:
    if type(k) is keras.layers.Dropout:
        model.layers.remove(k)

In [8]:
model.save('handWritten.h5')

In [9]:
import coremltools
# Core ML has 2 set of APIs Vision API and Natural Language API
output_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
scale = 1/255.
coreml_model = coremltools.converters.keras.convert('./handWritten.h5',
                                                    input_names='image',
                                                    image_input_names='image',
                                                    output_names='output',
                                                    class_labels= output_labels,
                                                    image_scale=scale)
coreml_model.author = 'Gerardo Lopez Falcon'
coreml_model.license = 'MIT'
coreml_model.short_description = 'Model to classify hand written digit'
coreml_model.input_description['image'] = 'Grayscale image of hand written digit'
coreml_model.output_description['output'] = 'Predicted digit'

coreml_model.save('handWritten.mlmodel')

0 : conv2d_1_input, <keras.engine.topology.InputLayer object at 0x12d3a9810>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x12d3a9850>
2 : conv2d_1__activation__, <keras.layers.core.Activation object at 0x12f0949d0>
3 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x12d3ffbd0>
4 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x12d3ff0d0>
5 : conv2d_2__activation__, <keras.layers.core.Activation object at 0x12f3f4550>
6 : max_pooling2d_2, <keras.layers.pooling.MaxPooling2D object at 0x12d44cb10>
7 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x12e3622d0>
8 : conv2d_3__activation__, <keras.layers.core.Activation object at 0x12f107f10>
9 : max_pooling2d_3, <keras.layers.pooling.MaxPooling2D object at 0x12d469cd0>
10 : flatten_1, <keras.layers.core.Flatten object at 0x12e3adc90>
11 : dense_1, <keras.layers.core.Dense object at 0x12e39f710>
12 : dense_1__activation__, <keras.layers.core.Activation object at 0x12eb35890>
13 : dense_2, <ker

- [Creating an IOS app with Core ML from scratch!] https://medium.com/towards-data-science/creating-an-ios-app-with-core-ml-from-scratch-b9e13e8af9cb
- [Now make your iOS apps intelligent!] https://medium.com/@aniket.ghode/now-make-your-ios-apps-intelligent-50c24903ffed